In [10]:
import os
import subprocess
import threading
import time

# Set environment variable we need to support dual-GPU on Cirrus
os.environ["NCCL_P2P_LEVEL"] = "NVL"

def run_vllm_server():
    subprocess.run([
        "vllm", "serve", "Qwen/Qwen3-32B-AWQ",
        "--host", "0.0.0.0",
        "--port", "8000",
        "--tensor-parallel-size", "2",
        "--trust-remote-code",
        "--gpu-memory-utilization", "0.4",
        "--enforce-eager",
        "--task", "embed"
    ])

# Start server in daemon thread
server_thread = threading.Thread(target=run_vllm_server, daemon=True)
server_thread.start()

## give server time to start up.

import time
# Pause execution for 100 seconds
time.sleep(200)

INFO 06-07 04:15:58 [__init__.py:243] Automatically detected platform cuda.
INFO 06-07 04:16:02 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-07 04:16:02 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-07 04:16:02 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-07 04:16:03 [api_server.py:1289] vLLM API server version 0.9.0.1
INFO 06-07 04:16:03 [cli_args.py:300] non-default args: {'host': '0.0.0.0', 'task': 'embed', 'trust_remote_code': True, 'enforce_eager': True, 'tensor_parallel_size': 2, 'gpu_memory_utilization': 0.4}
WARNING 06-07 04:16:14 [config.py:907] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 06-07 04:16:14 [arg_utils.py:1583] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 06-07 04:16:14 [arg_

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:04<00:13,  4.64s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:13<00:13,  6.86s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:21<00:07,  7.69s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:29<00:00,  7.84s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:29<00:00,  7.45s/it]



INFO 06-07 04:17:02 [default_loader.py:280] Loading weights took 29.98 seconds
(VllmWorkerProcess pid=14061) INFO 06-07 04:17:02 [default_loader.py:280] Loading weights took 30.20 seconds
INFO 06-07 04:17:02 [model_runner.py:1202] Model loading took 8.3324 GiB and 31.055884 seconds
(VllmWorkerProcess pid=14061) INFO 06-07 04:17:02 [model_runner.py:1202] Model loading took 8.3324 GiB and 31.096897 seconds
INFO 06-07 04:17:03 [api_server.py:1336] Starting vLLM API server on http://0.0.0.0:8000
INFO 06-07 04:17:03 [launcher.py:28] Available routes are:
INFO 06-07 04:17:03 [launcher.py:36] Route: /openapi.json, Methods: GET, HEAD
INFO 06-07 04:17:03 [launcher.py:36] Route: /docs, Methods: GET, HEAD
INFO 06-07 04:17:03 [launcher.py:36] Route: /docs/oauth2-redirect, Methods: GET, HEAD
INFO 06-07 04:17:03 [launcher.py:36] Route: /redoc, Methods: GET, HEAD
INFO 06-07 04:17:03 [launcher.py:36] Route: /health, Methods: GET
INFO 06-07 04:17:03 [launcher.py:36] Route: /load, Methods: GET
INFO 06-0

INFO:     Started server process [13556]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [ ]:
## wait for output above to print routes, ending with: 
## INFO:     Application startup complete.


In [2]:
## NOTE!  You must wait until the log above finishes and not just the cell.
## Connect to the local model
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
                 model = "Qwen/Qwen3-32B-AWQ",
                 api_key = "EMPTY",
                 base_url = "http://localhost:8000/v1",
)

## test that we can do embeddings
from langchain_core.vectorstores import InMemoryVectorStore
vectorstore = InMemoryVectorStore.from_texts(["test text"], embedding=embedding)

INFO 06-07 04:02:50 [logger.py:42] Received request embd-32a68fa8f24a4855b090f66f426e61c4-0: prompt: ' product down', params: PoolingParams(dimensions=None, additional_metadata=None), prompt_token_ids: [1985, 1495], prompt_embeds shape: None, lora_request: None, prompt_adapter_request: None.
INFO 06-07 04:02:50 [engine.py:316] Added request embd-32a68fa8f24a4855b090f66f426e61c4-0.
INFO 06-07 04:02:52 [metrics.py:486] Avg prompt throughput: 0.2 tokens/s, Avg generation throughput: 0.1 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.
INFO:     127.0.0.1:37090 - "POST /v1/embeddings HTTP/1.1" 200 OK


In [4]:
import os
import requests
import zipfile
import pathlib
from langchain_community.document_loaders import PyPDFLoader

def download_and_unzip(url, output_dir):
    response = requests.get(url)
    zip_file_path = os.path.basename(url)
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    os.remove(zip_file_path)

def pdf_loader(path):
    all_documents = []
    docs_dir = pathlib.Path(path)
    for file in docs_dir.iterdir():
        loader = PyPDFLoader(file)
        documents = loader.load()
        all_documents.extend(documents)
    return all_documents


download_and_unzip("https://minio.carlboettiger.info/public-data/hwc.zip", 'hwc')
docs = pdf_loader('hwc/')

In [9]:
# Set up the Chat model from one of the NRP models

import os
api_key = os.getenv("OPENAI_API_KEY")

# see `curl -H "Authorization: Bearer $OPENAI_API_KEY" https://llm.nrp-nautilus.io/v1/models`
models = {"llama3": "llama3-sdsc", 
          "deepseek-small": "DeepSeek-R1-Distill-Qwen-32B",
          "deepseek": "deepseek-r1-qwen-qualcomm",
          "gemma3": "gemma3",
          "phi3": "phi3",
          "olmo": "olmo"
         }

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = models["gemma3"], 
                 api_key = api_key, 
                 base_url = "https://llm.nrp-nautilus.io",  
                 temperature=0)

# Embedding model from NRP usually times out.
#embedding = OpenAIEmbeddings(model = "embed-mistral", api_key = api_key, base_url = "https://llm.nrp-nautilus.io")


In [ ]:
# Build a retrival agent
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
# slow part here, runs on remote GPU
from langchain_core.vectorstores import InMemoryVectorStore
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
prompt = "I live in Tanzania and am having issues with lions breaking into my boma and preying on cattle. What interventions might work best for me?"
results = rag_chain.invoke({"input": prompt})
results

In [ ]:
prompt = "What are the most cost-effective prevention methods for elephants raiding my crops?"

results = rag_chain.invoke({"input": prompt})
results

In [ ]:
rag_chain.invoke({"input": 
                  "I have a small herd of goats and cattle and I am worried about jaguars preying on them. What preventative measures can I take?"
                 })

In [ ]:
rag_chain.invoke({"input": "I am trying to prevent coyotes from eating the calves of my free-range cattle. What may work best?"})

In [ ]:
rag_chain.invoke({"input": "We have major issues with deer raiding our large agricultural fields. Is there anything I can try to prevent this that won’t break the bank?"})

In [ ]:
rag_chain.invoke({"input": "We live in a suburban area and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community? We don’t want to have to get rid of our fruit trees…"})

In [ ]:
prompt = "What cattle husbandry strategies might be helpful to prevent conflict if we live in wolf country?"

rag_chain.invoke({"input": prompt})

In [ ]:
## DRAFT exploring other embedding databases

import os
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import gc
import torch

# Option 1: FAISS (Facebook AI Similarity Search) - Most memory efficient
def create_faiss_vectorstore(splits, embedding, persist_directory="./faiss_db", batch_size=100):
    """
    Create FAISS vector store with batched processing to minimize GPU RAM usage
    """
    os.makedirs(persist_directory, exist_ok=True)
    
    # Process documents in batches to avoid GPU memory overflow
    vectorstore = None
    
    for i in range(0, len(splits), batch_size):
        batch = splits[i:i + batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(splits) + batch_size - 1)//batch_size}")
        
        if vectorstore is None:
            # Create initial vectorstore with first batch
            vectorstore = FAISS.from_documents(
                documents=batch,
                embedding=embedding
            )
        else:
            # Add subsequent batches to existing vectorstore
            batch_vectorstore = FAISS.from_documents(
                documents=batch,
                embedding=embedding
            )
            vectorstore.merge_from(batch_vectorstore)
            
            # Clean up temporary vectorstore
            del batch_vectorstore
        
        # Force garbage collection and clear GPU cache if using CUDA
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Save to disk
    vectorstore.save_local(persist_directory)
    print(f"Vector store saved to {persist_directory}")
    
    return vectorstore

def load_faiss_vectorstore(embedding, persist_directory="./faiss_db"):
    """Load existing FAISS vector store from disk"""
    return FAISS.load_local(
        persist_directory,
        embedding,
        allow_dangerous_deserialization=True  # Only if you trust the source
    )

# Option 2: Chroma - Persistent SQLite-based storage
def create_chroma_vectorstore(splits, embedding, persist_directory="./chroma_db", batch_size=100):
    """
    Create Chroma vector store with batched processing
    """
    # Initialize Chroma with persistence
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    
    # Add documents in batches
    for i in range(0, len(splits), batch_size):
        batch = splits[i:i + batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(splits) + batch_size - 1)//batch_size}")
        
        vectorstore.add_documents(batch)
        
        # Force garbage collection and clear GPU cache
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Persist to disk
    vectorstore.persist()
    print(f"Vector store persisted to {persist_directory}")
    
    return vectorstore

def load_chroma_vectorstore(embedding, persist_directory="./chroma_db"):
    """Load existing Chroma vector store from disk"""
    return Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )

# Option 3: Qdrant - High-performance vector database
def create_qdrant_vectorstore(splits, embedding, collection_name="documents", 
                            path="./qdrant_db", batch_size=100):
    """
    Create Qdrant vector store with local file-based storage
    """
    # Initialize local Qdrant client
    client = QdrantClient(path=path)
    
    # Get embedding dimension (embed a sample text)
    sample_embedding = embedding.embed_query("sample text")
    embedding_dim = len(sample_embedding)
    
    # Create collection if it doesn't exist
    try:
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE)
        )
    except Exception as e:
        print(f"Collection might already exist: {e}")
    
    # Create vectorstore
    vectorstore = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embedding
    )
    
    # Add documents in batches
    for i in range(0, len(splits), batch_size):
        batch = splits[i:i + batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(splits) + batch_size - 1)//batch_size}")
        
        vectorstore.add_documents(batch)
        
        # Force garbage collection and clear GPU cache
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    print(f"Vector store created in {path}")
    return vectorstore

def load_qdrant_vectorstore(embedding, collection_name="documents", path="./qdrant_db"):
    """Load existing Qdrant vector store from disk"""
    client = QdrantClient(path=path)
    return Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embedding
    )


In [ ]:

# Usage examples:

# Replace your original code with one of these options:

# Option 1: FAISS (Recommended for most use cases)
vectorstore = create_faiss_vectorstore(
    splits=splits, 
    embedding=embedding, 
    persist_directory="./my_faiss_db",
    batch_size=50  # Adjust based on your GPU memory
)

# To load later:
# vectorstore = load_faiss_vectorstore(embedding, "./my_faiss_db")

# Option 2: Chroma (Good for development and moderate scale)
# vectorstore = create_chroma_vectorstore(
#     splits=splits,
#     embedding=embedding,
#     persist_directory="./my_chroma_db",
#     batch_size=50
# )

# Option 3: Qdrant (Best for production and very large scale)
# vectorstore = create_qdrant_vectorstore(
#     splits=splits,
#     embedding=embedding,
#     collection_name="my_documents",
#     path="./my_qdrant_db",
#     batch_size=50
# )

# Memory optimization settings
def optimize_gpu_memory():
    """Additional GPU memory optimization"""
    if torch.cuda.is_available():
        # Set memory fraction if needed
        torch.cuda.set_per_process_memory_fraction(0.8)  # Use 80% of GPU memory
        
        # Enable memory mapping for large tensors
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

# Call before processing if you have GPU memory issues
# optimize_gpu_memory()